# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [17]:
import os
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [8]:
# constants

MODEL_LLAMA = 'llama3.2:1b'
OLLAMA_API = "http://localhost:11434/v1"
HEADERS = {"Content-Type": "application/json"}

In [9]:
# initialise ollama interface
ollama = OpenAI(base_url=OLLAMA_API, api_key='ollama')

In [14]:
class TechnicalSpecialist:
    def __init__(self):
        self.system_prompt = "You are a technocal specialist in wide spectrum. \
            Your task is to answer to technical questions with deep explenation. \
            When you expleining try to be as simple as possible like you are describing it for child. \
            Respond in Markdown."
    
    def answer(self, question, with_stram = True):
        response = ollama.chat.completions.create(
            model = MODEL_LLAMA,
            messages=[
                {'role': 'system', 'content': self.system_prompt},
                {'role': 'user', 'content': question}
            ],
            stream=with_stram
        )
        return self.stream_result(response) if with_stram else display(Markdown(response.choices[0].message.content))
    
    def stream_result(self, stream):
        response = ""
        display_handle = display(Markdown(""), display_id=True)
        for chunk in stream:
            response += chunk.choices[0].delta.content or ''
            response = response.replace("```","").replace("markdown", "")
            update_display(Markdown(response), display_id=display_handle.display_id)

        




In [15]:
techGuru = TechnicalSpecialist()

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [18]:
techGuru.answer(question)

**Code Explanation: ` yield from` with `dict.get()`**

This is a Python code snippet that uses the `yield from` expression in combination with the dictionary lookup operator (`get`) to achieve something similar to a generator expression.

Let's break it down:

1. `{book.get("author") for book in books if book.get("author")}`:
	* This is a complex list comprehension that creates a new iterator by looping over each item `book` in the `books` dictionary.
	* It checks whether both `book["author"]` exists and has a value, then skips to the next iteration using the conditional statement `if book.get("author")`.
2. `yield from ...`: This is where things get interesting!

 Inside the list comprehension, this line uses another `for` loop (`dict.get()`), but it doesn't do the loop itself.

Here's what happens:

- It loops over each key-value pair in the dictionary.
- For each iteration, it checks using `dict.get()`, if both values are available (as keys or dictionaries). If they are not, you get an error.
- If `dict.get()` returns a value (not None), that value becomes a new item in the iterator.

When all iterations of this loop complete and no errors occurred, each successful execution is used to generate its corresponding element on the outer generator by assigning it's next result from the previous generator:

Here are some simplified examples of what `yield from` can produce:

python
# A simple example using a list comprehension
books = [
    {"author": "Author 1", "title": "Book"},
    {"title": "Book2"},
    # ... (skipped this one)
]
result_list =[i["author"] for i in books if hasattr(i, 'author')]
print(type(result_list)) # It will print a list

# Using yield from: more explicit example
d = {
    1 : True,
    2 : False,
}
results = []
for key, value in d.items():
    results.extend([value] * (key*3))
yfrom = iter(results)
print(next(yfrom)) # It will print "True"
next(yfrom) # Now it prints "False" three times


**Why use `yield from`?**

The main benefits of using `yield from` are:

- Improves Code Readability: The list comprehension syntax can make the code easier to read, especially when working with lists or other types of iterables.
- Promotes Generators: It encourages breaking down complex operations into smaller steps that produce a sequence of results. This makes your code more flexible and efficient.

**Important Security Note:** When using `dict.get()` multiple times within a small scope (e.g., as in the example), it can lead to potential dictionary exhaustion issues if you're dealing with large data sets.

This usage of `yield from` typically occurs when working with complex, nested iterables that might grow impractically long. It represents an efficient way to build up a sequence one item at a time without loading the entire dataset into memory at once.